# NLP Workshop Demo: Sentiment Analysis for Movie Reviews

## --- ALL CREDIT FOR THIS DEMO GOES TO KAGGLE'S WORD2VEC NLP TUTORIAL!! ---

## Setting up your system:
### packages: pandas, numpy, scipy, scikit-learn, Beautiful Soup, NLTK, Cython, gensim

## Data:
### Labeled data set: 50,000 IMDB movie reviews - a rating less than 5 is given a sentiment score of 0, and a                rating greater than 7 is given a sentiment score of 1

### Unlabeled dataset: 50,000 IMDB reviews without any rating labels

### Data fields: (1) id - unique ID of each review, (2) sentiment score of 0 or 1, (3) review - text of the review

# Part 1: Basic Bag of Words Model

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Import the pandas package, then use the "read_csv" function to read
# the labeled training data
import pandas as pd       
train = pd.read_csv("labeledTrainData.tsv", header=0, \
                    delimiter="\t", quoting=3)

In [ ]:
# Make sure that we read 25,000 rows and 3 columns
train.shape

In [ ]:
# View column names
train.columns.values

In [ ]:
# Look at the first review
print(train["review"][0])

# Data cleaning and preprocessing

## If you don't have beautiful soup installed, do (command line): sudo pip install BeautifulSoup4

In [ ]:
# Import BeautifulSoup into your workspace
from bs4 import BeautifulSoup             

# Initialize the BeautifulSoup object on a single movie review     
example1 = BeautifulSoup(train["review"][0])  

# Print the raw review and then the output of get_text(), for 
# comparison
print (train["review"][0])
print (example1.get_text())

## Dealing with punctuation, numbers and stopwords: NLTK and regular expressions

In [ ]:
import re
# Use regular expressions to do a find-and-replace
letters_only = re.sub("[^a-zA-Z]",           # The pattern to search for
                      " ",                   # The pattern to replace it with
                      example1.get_text() )  # The text to search
print (letters_only)

In [ ]:
# Tokenization
lower_case = letters_only.lower()        # Convert to lower case
words = lower_case.split()               # Split into words

# Remove "stop words":  Download text data sets, including stop words
### import nltk
### nltk.download().encode('ascii', 'ignore')

In [ ]:
import nltk
from nltk.corpus import stopwords # Import the stop word list
print (stopwords.words("english"))

In [ ]:
# Remove stop words from "words"
words = [w for w in words if not w in stopwords.words("english")]
print(words)

# Putting it all together
This is the main function that will conduct all the pre-processing in one go.

In [ ]:
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

## Demo

Cleaning one review. Next cell cleans all the reviews in go

In [ ]:
# Clean a review
# clean_review = review_to_words( train["review"][0] )
# print(clean_review)

In [ ]:
# Clean all reviews
# Get the number of reviews based on the dataframe column size
num_reviews = train["review"].size

# Initialize an empty list to hold the clean reviews
clean_train_reviews = []

# Loop over each review; create an index i that goes from 0 to the length
# of the movie review list 
for i in range( 0, num_reviews ):
    # Call our function for each one, and add the result to the list of
    # clean reviews
    clean_train_reviews.append( review_to_words( train["review"][i] ))

clean_train_reviews is an array of reviews,  each processed review is an element

## Creating Features using Bag of Words

In [ ]:
print("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

In [ ]:
# Examine the dimension of our feaeturized training data
print(train_data_features.shape)

In [ ]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print(vocab)

In [ ]:
# Print counts of each word in the vocabulary
import numpy as np

# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print (count, tag)

## A Random Forest Model for Sentiment Classification

In [ ]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features[0:20000,:], train["sentiment"][0:20000] )

In [ ]:
# Validation accuracy 
valid_predictions = forest.predict(train_data_features[20000:,:])
train_predictions = forest.predict(train_data_features[0:20000,:])
train_acc = np.mean(train_predictions == train["sentiment"][0:20000])
valid_acc = np.mean(valid_predictions == train["sentiment"][20000:])
print("The training accuracy is: ", train_acc, "\n", "The validation accuracy is: ", valid_acc)

In [ ]:
# Predicting the test data

# Read the test data
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", \
                   quoting=3 )

# Verify that there are 25,000 rows and 2 columns
print(test.shape)

# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = [] 

print("Cleaning and parsing the test set movie reviews...\n")
for i in range(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )

# Part 2: Word Vectors

In [ ]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))

In [ ]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download()   

In [ ]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('nltk:tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [ ]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

# Check how many sentences we have in total - should be around 850,000+
print(len(sentences))

In [ ]:
print(sentences[0])

In [ ]:
print(sentences[1])

In [ ]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

## Exploring the Model Results

In [ ]:
model.doesnt_match("man woman child kitchen".split())

In [ ]:
model.doesnt_match("france england germany berlin".split())

In [ ]:
model.doesnt_match("paris berlin london austria".split())

In [ ]:
model.most_similar("man")

In [ ]:
model.most_similar("queen")

In [ ]:
model.most_similar("awful")